# Simulación del Truco

Las **cartas** deben registra el numero, palo y valor de la carta. 
El valor debe reflejar la jerarquía de las cartas en el juego del truco.

In [5]:
def calcular_valor(numero, palo):
    valores = {
        (1, 'espada'): 14,
        (1, 'basto'): 13,
        (7, 'oro'): 12,
        (7, 'copa'): 11,
        3: 10,
        2: 9,
        1: 8,
        12: 7,
        11: 6,
        10: 5,
        7: 4,
        6: 3,
        5: 2,
        4: 1
    }
    return valores.get((numero, palo), valores.get(numero, 0))

emojis = {
    'oro': '🌞', 
    'copa': '🍷', 
    'espada': '🗡️', 
    'basto': '🥖'
}

class Carta:
    def __init__(self, numero, palo):
        self.numero = numero
        self.palo = palo
        self.valor = calcular_valor(numero, palo)

    def __repr__(self):
        return f"{self.numero} de {emojis[self.palo]}"

    def __gt__(self, otro):
        return self.valor > otro.valor
    
c = Carta(1, "oro")


**Mazo**: Representan las cartas españolas de 1 al 12, en los palos de oro, copa, espada y basto.

Debe existir una función que permita barajar las cartas.
y otra función que permita repartir las cartas a los jugadores.

In [6]:
import random

class Mazo:
    def __init__(self):
        self.cartas = [
            Carta(numero, palo) for numero in range(1, 13) 
            for palo in ["oro", "copa", "espada", "basto"] 
            if numero != 8 and numero != 9
        ]
    
    def mezclar(self):
        random.shuffle(self.cartas)

    def repartir(self, cantidad):
        return [self.cartas.pop() for _ in range(cantidad)] if len(self.cartas) >= cantidad else []


m = Mazo()
m.mezclar()
print(m.repartir(3))

[10 de 🥖, 6 de 🍷, 5 de 🥖]


**Jugador**: Deben tener un nombre y las cartas que tiene en la mano. Debe acumular los puntos ganados.

Nota: Para simplificar el problema, el jugador elige la carta a jugar de forma aleatoria.

In [7]:
class Jugador:
    def __init__(self, nombre):
        self.nombre = nombre
        self.cartas_en_mano = []
        self.puntos = 0

    def __repr__(self):
        return self.nombre

    def recibir_cartas(self, cartas):
        self.cartas_en_mano = cartas

    def jugar_carta(self):
        return self.cartas_en_mano.pop(random.randint(0, len(self.cartas_en_mano)-1)) if self.cartas_en_mano else None

j = Jugador("Juan")
j.recibir_cartas(m.repartir(3))
print(j.jugar_carta())

7 de 🍷


**Partida**: Debe tener dos jugadores, un mazo de cartas y un puntaje. Debe permitir jugar una carta por ronda y definir el ganador de la ronda. Debe permitir jugar hasta que se terminen las cartas del mazo.

In [9]:
class Partida:
    def __init__(self, jugador1, jugador2):
        self.jugadores = [jugador1, jugador2]
        self.mazo = Mazo()

    def jugar_ronda(self):
        return [[j.jugar_carta() for j in self.jugadores] for _ in range(3)]
    
    def evaluar_jugada(self, jugadas):
        ganadas = [
            sum(1 for c1, c2 in jugadas if c1 > c2),
            sum(1 for c1, c2 in jugadas if c2 > c1)
        ]
        
        for ganadas, jugador in zip(ganadas, self.jugadores):
            if ganadas >= 2:
                jugador.puntos += 1
                print(f"{jugador} gana la ronda")

    def puntajes(self):
        print(f"\nPuntos:")
        for jugador in self.jugadores:
            print(f" - {jugador}: {jugador.puntos}")
        print('/-/' * 20)

    def jugar(self):
        ronda = 0
        
        while all(j.puntos < 15 for j in self.jugadores):
            self.mazo = Mazo()
            self.mazo.mezclar()
            ronda += 1
            print(f"Ronda {ronda} {[jugador for jugador in self.jugadores]}")

            for jugador in self.jugadores:
                jugador.recibir_cartas(self.mazo.repartir(3))
            
            jugadas = self.jugar_ronda()
            for jugada in jugadas:
                print(' ', jugada)
            
            self.evaluar_jugada(jugadas)
            self.puntajes()

        for jugador in self.jugadores:
            if jugador.puntos >= 15:
                print(f"{jugador} gana la partida")

p = Partida(Jugador("Juan"), Jugador("Pedro"))
p.jugar()

Ronda 1 [Juan, Pedro]
  [12 de 🌞, 11 de 🥖]
  [7 de 🌞, 10 de 🌞]
  [3 de 🍷, 4 de 🥖]
Juan gana la ronda

Puntos:
 - Juan: 1
 - Pedro: 0
/-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-/
Ronda 2 [Juan, Pedro]
  [4 de 🍷, 10 de 🗡️]
  [2 de 🌞, 5 de 🗡️]
  [1 de 🗡️, 12 de 🍷]
Juan gana la ronda

Puntos:
 - Juan: 2
 - Pedro: 0
/-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-/
Ronda 3 [Juan, Pedro]
  [10 de 🗡️, 2 de 🍷]
  [5 de 🍷, 3 de 🗡️]
  [1 de 🗡️, 3 de 🌞]
Pedro gana la ronda

Puntos:
 - Juan: 2
 - Pedro: 1
/-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-/
Ronda 4 [Juan, Pedro]
  [12 de 🗡️, 1 de 🥖]
  [4 de 🗡️, 2 de 🗡️]
  [6 de 🥖, 4 de 🍷]
Pedro gana la ronda

Puntos:
 - Juan: 2
 - Pedro: 2
/-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-/
Ronda 5 [Juan, Pedro]
  [6 de 🗡️, 10 de 🥖]
  [1 de 🌞, 5 de 🍷]
  [7 de 🥖, 11 de 🗡️]
Pedro gana la ronda

Puntos:
 - Juan: 2
 - Pedro: 3
/-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-//-/
Ronda 6 [Juan, Pedro]
